# Exp 2. This is a notebook using GPT to generate the cypher  to communicate with the database

Ref: https://github.com/tomasonjo/blogs/blob/master/llm/generic_cypher_gpt4.ipynb

Make a imporvement by fetching the patterns in the cypther query

In [1]:
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
import openai
from dotenv import load_dotenv
import os
import re

load_dotenv()
neo4j_url = os.getenv('NEO4J_URL')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')
openai_key = os.getenv('OPENAI_KEY')


In [2]:
node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""

rel_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {type: nodeLabels, properties: properties} AS output
"""

rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
RETURN {source: label, relationship: property, target: other} AS output
"""

In [3]:
def schema_text(node_props, rel_props, rels):
    return f"""
  This is the schema representation of the Neo4j database.
  Node properties are the following:
  {node_props}
  Relationship properties are the following:
  {rel_props}
  Relationship point from source to target nodes
  {rels}
  Make sure to respect relationship types and directions
  """


def fetch_cypher_query(text):
    match = re.search('```\n(.+?)\n```', text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return False

class Neo4jGPTQuery:
    def __init__(self, url, user, password, openai_api_key):
        self.driver = GraphDatabase.driver(url, auth=(user, password))
        openai.api_key = openai_api_key
        # construct schema
        self.schema = self.generate_schema()

    def generate_schema(self):
        node_props = self.query_database(node_properties_query)
        rel_props = self.query_database(rel_properties_query)
        rels = self.query_database(rel_query)
        return schema_text(node_props, rel_props, rels)

    def refresh_schema(self):
        self.schema = self.generate_schema()

    def get_system_message(self):
        return f"""
        Task: Generate Cypher queries to query a Neo4j graph database based on the provided schema definition.
        Instructions:
        Use only the provided relationship types and properties.
        Do not use any other relationship types or properties that are not provided.
        If you cannot generate a Cypher statement based on the provided schema, explain the reason to the user.
        Schema:
        {self.schema}

        Note: Do not include any explanations or apologies in your responses.
        """

    def query_database(self, neo4j_query, params={}):
        if fetch_cypher_query(neo4j_query):
            neo4j_query = fetch_cypher_query(neo4j_query)
            
        with self.driver.session() as session:
            result = session.run(neo4j_query, params)
            output = [r.values() for r in result]
            output.insert(0, result.keys())
            return output

    def construct_cypher(self, question, history=None):
        messages = [
            {"role": "system", "content": self.get_system_message()},
            {"role": "user", "content": question},
        ]
        # Used for Cypher healing flows
        if history:
            messages.extend(history)

        completions = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.0,
            max_tokens=2000,
            messages=messages
        )
        return completions.choices[0].message.content

    def run(self, question, history=None, retry=True):
        cypher = self.construct_cypher(question, history)
        try:
            return self.query_database(cypher)
        
        # Self-healing flow
        except CypherSyntaxError as e:
            # If out of retries
            if not retry:
                return "Invalid Cypher syntax"
            
            # Self-healing Cypher flow by
            # providing specific error to GPT-4
            print("Retrying")
            return self.run(
                question,
                [
                    {"role": "assistant", "content": cypher},
                    {
                        "role": "user",
                        "content": f"""This query returns an error: {str(e)} 
                        Please give me a improved query that works WITHOUT any explanations or apologies. 
                        Just output the Neo4j cypther!!!!""",
                    },
                ],
                retry=False
            )

In [4]:
ps_db = Neo4jGPTQuery(
    url=neo4j_url,
    user=neo4j_user,
    password=neo4j_password,
    openai_api_key=openai_key,
)

In [5]:
ps_db.run("""
What are the nodes in the databse?
""")

Retrying


[['n'],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a6:0' labels=frozenset({'Project_risk'}) properties={'name': 'Poor_project_execution', 'id': 1}>],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a6:1' labels=frozenset({'Project_risk'}) properties={'name': 'Non-Delivery_of_benefits', 'id': 2}>],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a6:2' labels=frozenset({'Project_risk'}) properties={'name': 'Business_disruption_', 'id': 3}>],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a6:3' labels=frozenset({'Project_risk'}) properties={'name': 'Slowing_digitisation_pace', 'id': 4}>],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a6:4' labels=frozenset({'Project_risk'}) properties={'name': 'Slowed_technological_change', 'id': 5}>],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a6:5' labels=frozenset({'Mitigation'}) properties={'name': 'Executive_sponsorship', 'id': 6}>],
 [<Node element_id='4:d6043383-25ec-44a0-921e-2d58e02b99a

In [6]:
ps_db.run("""
How many types of strategy theme?
""")

Retrying


[['COUNT(s)'], [8]]

In [7]:
ps_db.run("""
How many types of Q4_triggers?
""")

Retrying


[['COUNT(DISTINCT q)'], [5]]

In [8]:
ps_db.run("""
How many types of Mitigation and give the list of their names?
""")

[['num_types', 'mitigation_types'],
 [7,
  ['Executive_sponsorship',
   'Steering_group',
   'Regular_progress_updates',
   'Broad_stakeholder_representation',
   'Link_with_digital_programme_team',
   'Appropriate_personnel',
   'Monitor_volume_of_change_programmes']]]

In [9]:
ps_db.run("""
"What types of values that Unilever_create?"
""")

Retrying


[['n.name', 'n.id'],
 ['1._Consumer_benefits', 79],
 ['2._Top_&_bottom_line_growth', 80],
 ['3._Improved_health_and_well_being', 81],
 ['4._reduced_Environmental_impact', 82],
 ['5._Enhanced_livelihoods', 83]]